# Import packages

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

# Downloading data 

In [2]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareclimate_data/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]

In [4]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 2.74 s, sys: 3.09 s, total: 5.84 s
Wall time: 48.5 s


## Extract contents of zipped file

In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(f'{output_directory}/data')

CPU times: user 17.3 s, sys: 3.03 s, total: 20.4 s
Wall time: 20.8 s


## Remove unused file

In [6]:
unused_file = os.path.join(
    output_directory,
    "data/observed_daily_rainfall_SYD.csv")
if os.path.exists(unused_file):
    os.remove(unused_file)

# Combine data CSVs

In [7]:
%%time
files = glob.glob('figshareclimate_data/data/*.csv')
df = pd.concat(
    (pd.read_csv(file, index_col=0, parse_dates=['time'])
     .assign(model=re.findall(r'[^\/]+(?=_daily_rainfall_NSW\.)', file)[0])
     for file in files)
)
df.to_csv("figshareclimate_data/combined_data.csv")

CPU times: user 8min 24s, sys: 18.5 s, total: 8min 42s
Wall time: 8min 50s


In [8]:
print(df.shape)

(62467843, 6)


In [9]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62467843 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float64
 1   lat_max        float64
 2   lon_min        float64
 3   lon_max        float64
 4   rain (mm/day)  float64
 5   model          object 
dtypes: float64(5), object(1)
memory usage: 3.3+ GB


In [11]:
df.dtypes

lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

In [15]:
# check size of full csv on disk
#from nicenumber import nicenumber as nn
#nn.to_human(p_combined.stat().st_size, prec=1, family='filesize')

## Combine data csv on different machines

| Team Member            | Operating System | RAM | Processor | Is SSD | Time taken |
|:----------------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Kingslin Lv            | Windows          | 16GB|    i7      | Yes     | 15min 15s   |
| Sufang Tan             |     |           |        |            |
| Amir Abbas Shojakhani  |Mac OS     |16GB           |i7       |Yes           |08min 50s

# EDA